In [1]:
pip install sounddevice


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pydub


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install librosa soundfile numpy


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyttsx3


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gtts


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sounddevice as sd
from scipy.io.wavfile import write

In [7]:
import librosa
import soundfile as sf
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
import os
import pyttsx3
import time
import threading

In [8]:
from gtts import gTTS

In [10]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to ask a question with a scary voice and simultaneous music and input
def ask_with_scary_voice(text, music_file, timeout=15):
    """
    Ask a question in a scary deep voice with background music and timeout for user input.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    :param timeout: Timeout in seconds for user input.
    :return: User's response to the question or None if timed out.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()  # Wait until TTS finishes speaking

    # Get the user's input with a timeout
    user_response = [None]

    def get_input():
        user_response[0] = input("Enter your answer: ")

    input_thread = threading.Thread(target=get_input)
    input_thread.start()
    input_thread.join(timeout)

    # Stop the background music
    stop_event.set()
    music_thread.join()

    if input_thread.is_alive():
        print("Time's up! No answer received.")
        return None

    return user_response[0]


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"
    
    # Step 1: Ask for the user's name in their voice
    name = ask_with_scary_voice("Say out your name aloud : ", music_filename)
    print("Say out your name aloud : ")
    record_voice(input_filename, duration=10)
    play_audio(music_filename)
    
    # Step 2: Modulate the name into a scary voice and ask the first question
    #print("Processing your voice...")
    #blend_voice_with_music(input_filename, music_filename, output_filename)
    
    # Step 3: Ask the questions with simultaneous music and delay between questions
    fear = ask_with_scary_voice("What is your greatest fear?", music_filename)
    if fear is not None:
        print(f"Answer recorded: {fear}")
    else:
        print("No answer received for the first question.")
    
    # Wait 5 seconds before asking the next question

    hide_place = ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    if hide_place is not None:
        print(f"Answer recorded: {hide_place}")
    else:
        print("No answer received for the second question.")

    print("Run Away!")


Say out your name aloud : 
Using input device: MacBook Pro Microphone
Recording... Speak now!
Enter your answer: ghosts
Answer recorded: ghosts
Enter your answer: cupboard
Answer recorded: cupboard
Run Away!


In [23]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to ask a question with a scary voice and simultaneous music and input
def ask_with_scary_voice(text, music_file, timeout=15):
    """
    Ask a question in a scary deep voice with background music and timeout for user input.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    :param timeout: Timeout in seconds for user input.
    :return: User's response to the question or None if timed out.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()  # Wait until TTS finishes speaking

    # Get the user's input with a timeout
    user_response = [None]

    def get_input():
        user_response[0] = input("Enter your answer: ")

    input_thread = threading.Thread(target=get_input)
    input_thread.start()
    input_thread.join(timeout)

    # Stop the background music
    stop_event.set()
    music_thread.join()

    if input_thread.is_alive():
        print("Time's up! No answer received.")
        return None

    return user_response[0]


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Say your name aloud.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions with music playing simultaneously and handle timeout
    fear = ask_with_scary_voice("What is your greatest fear?", music_filename)
    if fear is not None:
        print(f"Answer recorded: {fear}")
    else:
        print("No answer received for the first question.")

    # Add delay before the next question
    time.sleep(5)

    hide_place = ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    if hide_place is not None:
        print(f"Answer recorded: {hide_place}")
    else:
        print("No answer received for the second question.")

    print("Thank you for answering!")


Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...
Enter your answer: ghosts
Answer recorded: ghosts
Enter your answer: cupboard
Answer recorded: cupboard
Thank you for answering!


In [22]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to ask a question with a scary voice and input timeout
def ask_with_scary_voice(text, music_file, timeout=15):
    """
    Ask a question in a scary deep voice with background music and timeout for user input.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    :param timeout: Timeout in seconds for user input.
    :return: User's response to the question or None if timed out.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()  # Wait until TTS finishes speaking

    # Get the user's input with a timeout
    user_response = [None]

    def get_input():
        user_response[0] = input("Enter your answer: ")

    input_thread = threading.Thread(target=get_input)
    input_thread.start()
    input_thread.join(timeout)

    # Stop the background music
    stop_event.set()
    music_thread.join()

    if input_thread.is_alive():
        print("Time's up! No answer received.")
        return None

    return user_response[0]


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Please say your name.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions with a timeout and delay
    fear = ask_with_scary_voice("What is your greatest fear?", music_filename)
    if fear is not None:
        print(f"Answer recorded: {fear}")
    else:
        print("No answer received for the first question.")
    # Add 5-second delay
    time.sleep(5)

    hide_place = ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    if hide_place is not None:
        print(f"Answer recorded: {hide_place}")
    else:
        print("No answer received for the second question.")
    # Add 5-second delay
    time.sleep(5)

    print("Thank you for answering!")

Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...
Enter your answer: Ghosts
Answer recorded: Ghosts
Enter your answer: Cupboard
Answer recorded: Cupboard
Thank you for answering!


In [21]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to ask a question with a scary voice and input timeout
def ask_with_scary_voice(text, music_file, timeout=15):
    """
    Ask a question in a scary deep voice with background music and timeout for user input.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    :param timeout: Timeout in seconds for user input.
    :return: User's response to the question or None if timed out.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()  # Wait until TTS finishes speaking

    # Get the user's input with a timeout
    user_response = [None]

    def get_input():
        user_response[0] = input("Enter your answer: ")

    input_thread = threading.Thread(target=get_input)
    input_thread.start()
    input_thread.join(timeout)

    # Stop the background music
    stop_event.set()
    music_thread.join()

    if input_thread.is_alive():
        print("Time's up! No answer received.")
        return None

    return user_response[0]


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Say your name aloud.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions with a timeout
    fear = ask_with_scary_voice("What is your greatest fear?", music_filename)
    if fear is not None:
        print(f"Answer recorded: {fear}")
    else:
        print("No answer received for the first question.")

    hide_place = ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    if hide_place is not None:
        print(f"Answer recorded: {hide_place}")
    else:
        print("No answer received for the second question.")

    print("Thank you for answering!")


Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...
Enter your answer: ghosts
Answer recorded: ghosts
Enter your answer: cupboard
Answer recorded: cupboard
Thank you for answering!


In [20]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to ask a question with a scary voice and wait for user input
def ask_with_scary_voice(text, music_file):
    """
    Ask a question in a scary deep voice with background music and wait for the user's answer.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    :return: User's response to the question.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate further
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()  # Wait until TTS finishes speaking

    # Stop the background music
    stop_event.set()
    music_thread.join()

    # Get the user's input
    return input("Enter your answer: ")


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Please say your name.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions in sequence and wait for answers
    fear = ask_with_scary_voice("What is your greatest fear?", music_filename)
    print(f"Answer recorded: {fear}")

    hide_place = ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    print(f"Answer recorded: {hide_place}")

    print("Thank you for answering!")


Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...


KeyboardInterrupt: 

In [11]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to play music in the background
def play_background_music(music_file, stop_event):
    """
    Play background music in a loop until stop_event is set.
    :param music_file: Path to the background music file.
    :param stop_event: A threading event to stop the music playback.
    """
    music_audio, sr = librosa.load(music_file, sr=None)
    while not stop_event.is_set():
        sd.play(music_audio, samplerate=sr)
        sd.wait()


# Function to modulate text into a scary deep voice
def ask_with_scary_voice(text, music_file):
    """
    Ask a question in a scary deep voice with background music.
    :param text: The question to ask.
    :param music_file: Path to the background music file.
    """
    # Start background music in a separate thread
    stop_event = threading.Event()
    music_thread = threading.Thread(target=play_background_music, args=(music_file, stop_event))
    music_thread.start()

    # Ask the question using text-to-speech
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()

    # Stop the background music
    stop_event.set()
    music_thread.join()


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Please say your name.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions in a scary deep voice with background music
    ask_with_scary_voice("What is your greatest fear?", music_filename)
    fear = input("Enter your answer: ")

    ask_with_scary_voice("What is your favorite place to hide?", music_filename)
    hide_place = input("Enter your answer: ")

    print("Thank you for answering!")


Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...
Enter your answer: ghosts


KeyboardInterrupt: 

In [10]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.9, music_volume=0.1):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to modulate text into a scary deep voice
def ask_with_scary_voice(text):
    """
    Ask a question in a scary deep voice using text-to-speech.
    """
    engine = pyttsx3.init()
    engine.setProperty('rate', 70)  # Slow down the speech rate further
    engine.setProperty('volume', 1.0)  # Max volume
    voices = engine.getProperty('voices')

    # Select a deeper voice if available
    for voice in voices:
        if "male" in voice.name.lower():  # Look for a male voice for a deeper tone
            engine.setProperty('voice', voice.id)
            break

    engine.say(text)
    engine.runAndWait()


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Please say your name.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions in a scary deep voice
    ask_with_scary_voice("What is your greatest fear?")
    fear = input("Enter your answer: ")

    ask_with_scary_voice("What is your favorite place to hide?")
    hide_place = input("Enter your answer: ")

    print("Thank you for answering!")

Please say your name.
Using input device: Shruti’s Phone  Microphone
Recording... Speak now!
Processing your voice...
Playing the scary voice...
Enter your answer: ghostss
Enter your answer: cupboard
Thank you for answering!


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import soundfile as sf
import numpy as np
import pyttsx3


# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find a valid input device
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            default_input_device = i
            break

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    # Set the default input device
    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    # Record the audio
    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()

    # Save the recorded audio
    write(filename, fs, (voice_data * 32767).astype('int16'))


# Function to blend voice with background music
def blend_voice_with_music(voice_file, music_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9, voice_volume=0.8, music_volume=0.2):
    """
    Blend recorded voice with a pre-downloaded haunting background music.
    """
    voice_audio, voice_sr = librosa.load(voice_file, sr=None)
    voice_audio, _ = librosa.effects.trim(voice_audio, top_db=30)
    pitched_voice = librosa.effects.pitch_shift(voice_audio, sr=voice_sr, n_steps=pitch_shift_steps)
    slowed_voice = librosa.effects.time_stretch(pitched_voice, rate=time_stretch_rate)
    normalized_voice = slowed_voice / np.max(np.abs(slowed_voice))
    music_audio, music_sr = librosa.load(music_file, sr=voice_sr)

    if len(music_audio) < len(normalized_voice):
        music_audio = np.tile(music_audio, int(np.ceil(len(normalized_voice) / len(music_audio))))[:len(normalized_voice)]
    else:
        music_audio = music_audio[:len(normalized_voice)]

    blended_audio = (voice_volume * normalized_voice) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))
    sf.write(output_file, blended_audio, voice_sr)


# Function to modulate text into a scary deep voice
def ask_with_scary_voice(text):
    """
    Ask a question in a scary deep voice using text-to-speech.
    """
    engine = pyttsx3.init()
    engine.setProperty('rate', 90)  # Slow the speech rate
    engine.setProperty('volume', 1.0)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)  # Use the first available voice
    engine.say(text)
    engine.runAndWait()


# Function to play audio
def play_audio(file_path):
    audio, sr = sf.read(file_path)
    sd.play(audio, samplerate=sr)
    sd.wait()


# Main program
if __name__ == "__main__":
    input_filename = "input.wav"
    music_filename = "haunting_music.wav"
    output_filename = "spooky_voice_with_music.wav"

    # Step 1: Ask for the user's name in their voice
    print("Please say your name.")
    record_voice(input_filename, duration=5)

    # Step 2: Modulate the name into a scary voice and ask the first question
    print("Processing your voice...")
    blend_voice_with_music(input_filename, music_filename, output_filename)
    print("Playing the scary voice...")
    play_audio(output_filename)

    # Step 3: Ask the questions in a scary deep voice
    ask_with_scary_voice("What is your greatest fear?")
    fear = input("Enter your answer: ")

    ask_with_scary_voice("What is your favorite place to hide?")
    hide_place = input("Enter your answer: ")

    print("Thank you for answering!")


In [11]:
import sounddevice as sd
sd.stop()

||PaMacCore (AUHAL)|| Error on line 2796: err='-50', msg=Unknown Error


In [15]:
# Function to record audio
def record_voice(filename, duration=10, fs=44100):
    """
    Record audio and save it as a WAV file.
    :param filename: Output file name.
    :param duration: Duration to record in seconds.
    :param fs: Sampling rate.
    """
    devices = sd.query_devices()
    default_input_device = None

    # Find the first valid input device (prioritize "MacBook Pro Microphone")
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            if "MacBook Pro Microphone" in device['name']:
                default_input_device = i
                break
            elif default_input_device is None:
                default_input_device = i

    if default_input_device is None:
        raise RuntimeError("No valid input device found for recording!")

    sd.default.device = [default_input_device, None]
    print(f"Using input device: {sd.query_devices()[default_input_device]['name']}")

    print("Recording... Speak now!")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()
    write(filename, fs, (voice_data * 32767).astype("int16"))
    print(f"Recording saved as {filename}")


# Function to synthesize "Hello [Name]" response
def synthesize_hello(name, output_file, fs=44100):
    """
    Synthesize "Hello [name]" as an audio waveform.
    :param name: The name to include.
    :param output_file: Output WAV file path.
    :param fs: Sampling rate.
    """
    import pyttsx3

    tts = pyttsx3.init()
    tts.setProperty("rate", 130)  # Set speech rate for clarity
    tts.setProperty("volume", 1.0)
    tts.save_to_file(f"Hello {name}", output_file)
    tts.runAndWait()


# Function to apply spooky voice effects
def apply_spooky_effects(input_file, output_file, pitch_shift_steps=-6, time_stretch_rate=0.9):
    """
    Apply spooky effects to audio.
    :param input_file: Input WAV file.
    :param output_file: Output WAV file.
    """
    audio, sr = librosa.load(input_file, sr=None)
    audio, _ = librosa.effects.trim(audio, top_db=30)
    pitched_audio = librosa.effects.pitch_shift(audio, sr=sr, n_steps=pitch_shift_steps)
    stretched_audio = librosa.effects.time_stretch(pitched_audio, rate=time_stretch_rate)
    normalized_audio = stretched_audio / np.max(np.abs(stretched_audio))
    sf.write(output_file, normalized_audio, sr)
    print(f"Spooky voice saved as {output_file}")


# Function to blend voice and haunting music
def blend_voice_with_music(voice_file, music_file, output_file, voice_volume=0.8, music_volume=0.2):
    """
    Blend the spooky voice with haunting background music.
    """
    voice_audio, sr = librosa.load(voice_file, sr=None)
    music_audio, _ = librosa.load(music_file, sr=sr)

    if len(music_audio) < len(voice_audio):
        music_audio = np.tile(music_audio, int(np.ceil(len(voice_audio) / len(music_audio))))
    music_audio = music_audio[: len(voice_audio)]

    blended_audio = (voice_volume * voice_audio) + (music_volume * music_audio)
    blended_audio = blended_audio / np.max(np.abs(blended_audio))

    sf.write(output_file, blended_audio, sr)
    print(f"Blended audio saved as {output_file}")


# Function to play audio
def play_audio(file_path):
    """
    Play an audio file.
    :param file_path: Path to the WAV file.
    """
    audio, sr = sf.read(file_path)
    print(f"Playing: {file_path}")
    sd.play(audio, samplerate=sr)
    sd.wait()
    print("Playback finished.")


# Main Program
if __name__ == "__main__":
    # File paths
    input_filename = "input.wav"
    hello_filename = "hello_name.wav"
    spooky_filename = "spooky_hello.wav"
    music_filename = "haunting_music.wav"  # Pre-downloaded music file
    output_filename = "final_output.wav"

    # Step 1: Ask for the name
    name = input("What is your name? ")

    # Step 2: Generate "Hello [name]"
    synthesize_hello(name, hello_filename)

    # Step 3: Apply spooky effects to "Hello [name]"
    apply_spooky_effects(hello_filename, spooky_filename)

    # Step 4: Blend spooky voice with haunting music
    blend_voice_with_music(spooky_filename, music_filename, output_filename)

    # Step 5: Play the final output
    play_audio(output_filename)


What is your name? Shruti


AttributeError: 'NSSpeechDriver' object has no attribute '_current_text'

In [16]:
import sounddevice as sd
import numpy as np

# Generate a test tone
fs = 44100  # Sampling rate
duration = 3  # seconds
frequency = 440.0  # A4 note
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
test_tone = 0.5 * np.sin(2 * np.pi * frequency * t)

# Set output device index (replace 4 with a valid index)
sd.default.device = [None, 4]  # Replace 4 with your output device index
try:
    sd.play(test_tone, samplerate=fs)
    sd.wait()
    print("Test tone played successfully.")
except Exception as e:
    print(f"Error during playback: {e}")


Error during playback: Error querying device 4
